In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
import numpy as np
from datetime import datetime
import random

In [ ]:
tournaments_df = pd.read_csv('tournaments.csv').set_index(['Name', 'Year'])
players_df = pd.read_csv('players.csv').set_index('Name')

matches_df = pd.read_csv('matches.csv')
matches_df = matches_df.drop(matches_df.columns[[0]], axis = 1)

In [ ]:
def find_ranking(row, player1 = True):
    name = row['Player 1'] if player1 else row['Player 2']
    tournament, year = row['Tournament Name'], row['Year']
    tournament_id = tournaments_df.loc[(tournament, year), 'Id'][0]
    try:
        url = 'https://www.atptour.com/en/scores/archive/%s/%d/%d/results' % (tournament.lower(), tournament_id, year)
        page = requests.get(url).text
    except Exception as e:
        return -1
    soup = BeautifulSoup(page)

    start_date = tournaments_df.loc[(tournament, year), 'Start Date'][0]
    try:
        name, player_id = name.lower().replace(" ", '-'), players_df.loc[name, 'Id']
    except Exception as e:
        print(e)
        return None
    ranking_page = 'https://www.atptour.com/en/players/%s/%s/rankings-history' % (name, player_id)
    historical_ranking = requests.get(ranking_page).text
    soup1 = BeautifulSoup(historical_ranking)

    dates, contents, format = [], [], '%Y.%m.%d'
    date_to_ranking = {}
    for tag in soup1.find_all('td'):
        tag = str(tag)[4:]
        content = tag[:tag.find('<')].strip()
        contents.append(content)
    for index, content in enumerate(contents):
        try:
            date = datetime.strptime(content, format)
            dates.append(date)
            ranking = int(contents[index + 1])
            date_to_ranking[content] = ranking
        except:
            continue

    ranking_date = -1
    start_date = datetime.strptime(start_date, '%Y-%m-%d')
    for index, date in enumerate(dates):
        if date <= start_date:
            ranking_date = date
            break
    try:
        ranking = date_to_ranking[datetime.strftime(ranking_date, format)]
    except:
        return -1
    return ranking

In [ ]:
starting_point = 0
for index, row in matches_df.iterrows():
    if index < starting_point:
        continue
    if (row['Player 1 Ranking'] == -1) | (row['Player 2 Ranking'] == -1):
        matches_df.at[index, 'Player 1 Ranking'] = find_ranking(row, player1 = True)
        matches_df.at[index, 'Player 2 Ranking'] = find_ranking(row, player1 = False)
        if (index % 50 == 0):
            print('saved')
            matches_df.to_csv('matches.csv')
    print(index)

In [ ]:
pass2 = matches_df[(matches_df['Player 1 Ranking'] == -1) | (matches_df['Player 2 Ranking'] == -1)].reset_index().drop('index', axis = 1)
pass2

In [ ]:
for index, row in pass2.iterrows():
    if (row['Player 1 Ranking'] == -1):
        pass2.at[index, 'Player 1 Ranking'] = find_ranking(row, player1 = True)
    if (row['Player 2 Ranking'] == -1):
        pass2.at[index, 'Player 2 Ranking'] = find_ranking(row, player1 = False)
    print(index)

In [ ]:
pass2